In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, np_utils
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model, model_from_json
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
import pandas as pd
import librosa
import glob 
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder


Using TensorFlow backend.


In [2]:
mylist= os.listdir(r'E:/SpeechEmotion/')
feeling_list=[]
for item in mylist:
    if item[3:-19]=='01':
        if item[6:-16]=='03' and int(item[18:-4])%2==0:
            feeling_list.append('female_happy')
        elif item[6:-16]=='03' and int(item[18:-4])%2==1:
            feeling_list.append('male_happy')
        elif item[6:-16]=='04' and int(item[18:-4])%2==0:
            feeling_list.append('female_sad')
        elif item[6:-16]=='04' and int(item[18:-4])%2==1:
            feeling_list.append('male_sad')
        elif item[6:-16]=='05' and int(item[18:-4])%2==0:
            feeling_list.append('female_angry')
        elif item[6:-16]=='05' and int(item[18:-4])%2==1:
            feeling_list.append('male_angry')
        
    

In [3]:
labels = pd.DataFrame(feeling_list)
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][3:-19]=='01' and (mylist[index][6:-16]=='03' or mylist[index][6:-16]=='04' or mylist[index][6:-16]=='05'):
        X, sample_rate = librosa.load('E:/SpeechEmotion/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
        

In [4]:
df3 = pd.DataFrame(df['feature'].values.tolist())
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf = shuffle(newdf)
rnewdf=rnewdf.fillna(0)
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [5]:
X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)
lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)


C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 216, 256)          1536      
_________________________________________________________________
activation_1 (Activation)    (None, 216, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 216, 128)          163968    
_________________________________________________________________
activation_2 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 27, 128)          

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
checkpoint = ModelCheckpoint(r"E:\demo1\speeash13.h5", monitor='val_accuracy',save_weights_only=True, mode='max', verbose=1,
                             save_best_only=True)
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=1000, validation_data=(x_testcnn, y_test),callbacks=[checkpoint])

fer_json = model.to_json()
with open("E:\demo1\speeash13.json", "w") as json_file:#0-62% 12->60.396
    json_file.write(fer_json)
  

Train on 475 samples, validate on 101 samples
Epoch 1/1000
475/475 [==============================] - ETA: 24s - loss: 0.2509 - accuracy: 0.937 - ETA: 13s - loss: 0.2445 - accuracy: 0.937 - ETA: 9s - loss: 0.2281 - accuracy: 0.937 - ETA: 7s - loss: 0.2223 - accuracy: 0.93 - ETA: 6s - loss: 0.2554 - accuracy: 0.91 - ETA: 5s - loss: 0.2504 - accuracy: 0.91 - ETA: 4s - loss: 0.2575 - accuracy: 0.92 - ETA: 4s - loss: 0.2424 - accuracy: 0.93 - ETA: 3s - loss: 0.2274 - accuracy: 0.94 - ETA: 3s - loss: 0.2157 - accuracy: 0.95 - ETA: 3s - loss: 0.2105 - accuracy: 0.95 - ETA: 3s - loss: 0.2159 - accuracy: 0.94 - ETA: 2s - loss: 0.2208 - accuracy: 0.94 - ETA: 2s - loss: 0.2440 - accuracy: 0.92 - ETA: 2s - loss: 0.2487 - accuracy: 0.91 - ETA: 2s - loss: 0.2588 - accuracy: 0.91 - ETA: 1s - loss: 0.2705 - accuracy: 0.90 - ETA: 1s - loss: 0.2703 - accuracy: 0.90 - ETA: 1s - loss: 0.2693 - accuracy: 0.90 - ETA: 1s - loss: 0.2713 - accuracy: 0.90 - ETA: 1s - loss: 0.2646 - accuracy: 0.91 - ETA: 1s - l

475/475 [==============================] - ETA: 2s - loss: 0.2336 - accuracy: 0.93 - ETA: 2s - loss: 0.2503 - accuracy: 0.87 - ETA: 2s - loss: 0.2144 - accuracy: 0.91 - ETA: 2s - loss: 0.2038 - accuracy: 0.93 - ETA: 2s - loss: 0.2047 - accuracy: 0.95 - ETA: 2s - loss: 0.1940 - accuracy: 0.94 - ETA: 2s - loss: 0.1914 - accuracy: 0.95 - ETA: 2s - loss: 0.1975 - accuracy: 0.96 - ETA: 2s - loss: 0.2025 - accuracy: 0.95 - ETA: 2s - loss: 0.1974 - accuracy: 0.96 - ETA: 2s - loss: 0.1923 - accuracy: 0.96 - ETA: 1s - loss: 0.1886 - accuracy: 0.96 - ETA: 1s - loss: 0.1918 - accuracy: 0.97 - ETA: 1s - loss: 0.1878 - accuracy: 0.97 - ETA: 1s - loss: 0.1990 - accuracy: 0.96 - ETA: 1s - loss: 0.2091 - accuracy: 0.96 - ETA: 1s - loss: 0.2129 - accuracy: 0.95 - ETA: 1s - loss: 0.2215 - accuracy: 0.95 - ETA: 1s - loss: 0.2202 - accuracy: 0.95 - ETA: 1s - loss: 0.2178 - accuracy: 0.95 - ETA: 0s - loss: 0.2130 - accuracy: 0.95 - ETA: 0s - loss: 0.2168 - accuracy: 0.95 - ETA: 0s - loss: 0.2178 - accuracy

475/475 [==============================] - ETA: 3s - loss: 0.2493 - accuracy: 0.93 - ETA: 3s - loss: 0.2189 - accuracy: 0.93 - ETA: 3s - loss: 0.2071 - accuracy: 0.95 - ETA: 3s - loss: 0.2439 - accuracy: 0.92 - ETA: 3s - loss: 0.2616 - accuracy: 0.92 - ETA: 3s - loss: 0.2557 - accuracy: 0.92 - ETA: 3s - loss: 0.2390 - accuracy: 0.92 - ETA: 2s - loss: 0.2558 - accuracy: 0.92 - ETA: 2s - loss: 0.2509 - accuracy: 0.92 - ETA: 2s - loss: 0.2435 - accuracy: 0.92 - ETA: 2s - loss: 0.2365 - accuracy: 0.93 - ETA: 2s - loss: 0.2270 - accuracy: 0.93 - ETA: 2s - loss: 0.2258 - accuracy: 0.93 - ETA: 2s - loss: 0.2278 - accuracy: 0.93 - ETA: 1s - loss: 0.2217 - accuracy: 0.93 - ETA: 1s - loss: 0.2165 - accuracy: 0.93 - ETA: 1s - loss: 0.2102 - accuracy: 0.94 - ETA: 1s - loss: 0.2214 - accuracy: 0.93 - ETA: 1s - loss: 0.2226 - accuracy: 0.93 - ETA: 1s - loss: 0.2209 - accuracy: 0.93 - ETA: 1s - loss: 0.2206 - accuracy: 0.94 - ETA: 1s - loss: 0.2191 - accuracy: 0.94 - ETA: 0s - loss: 0.2192 - accuracy

475/475 [==============================] - ETA: 3s - loss: 0.1899 - accuracy: 1.00 - ETA: 3s - loss: 0.1980 - accuracy: 1.00 - ETA: 3s - loss: 0.2082 - accuracy: 0.97 - ETA: 3s - loss: 0.2109 - accuracy: 0.96 - ETA: 3s - loss: 0.2078 - accuracy: 0.97 - ETA: 2s - loss: 0.2474 - accuracy: 0.93 - ETA: 2s - loss: 0.2396 - accuracy: 0.93 - ETA: 2s - loss: 0.2409 - accuracy: 0.92 - ETA: 2s - loss: 0.2296 - accuracy: 0.93 - ETA: 2s - loss: 0.2206 - accuracy: 0.94 - ETA: 2s - loss: 0.2170 - accuracy: 0.94 - ETA: 2s - loss: 0.2192 - accuracy: 0.94 - ETA: 2s - loss: 0.2176 - accuracy: 0.94 - ETA: 2s - loss: 0.2225 - accuracy: 0.94 - ETA: 1s - loss: 0.2276 - accuracy: 0.94 - ETA: 1s - loss: 0.2236 - accuracy: 0.94 - ETA: 1s - loss: 0.2224 - accuracy: 0.94 - ETA: 1s - loss: 0.2261 - accuracy: 0.94 - ETA: 1s - loss: 0.2252 - accuracy: 0.94 - ETA: 1s - loss: 0.2262 - accuracy: 0.94 - ETA: 1s - loss: 0.2202 - accuracy: 0.94 - ETA: 1s - loss: 0.2189 - accuracy: 0.94 - ETA: 0s - loss: 0.2197 - accuracy

475/475 [==============================] - ETA: 3s - loss: 0.2621 - accuracy: 0.93 - ETA: 3s - loss: 0.2192 - accuracy: 0.96 - ETA: 3s - loss: 0.2037 - accuracy: 0.97 - ETA: 3s - loss: 0.2226 - accuracy: 0.96 - ETA: 3s - loss: 0.2064 - accuracy: 0.97 - ETA: 3s - loss: 0.1920 - accuracy: 0.96 - ETA: 2s - loss: 0.1928 - accuracy: 0.97 - ETA: 2s - loss: 0.1913 - accuracy: 0.97 - ETA: 2s - loss: 0.1888 - accuracy: 0.97 - ETA: 2s - loss: 0.1930 - accuracy: 0.96 - ETA: 2s - loss: 0.2089 - accuracy: 0.95 - ETA: 2s - loss: 0.2043 - accuracy: 0.95 - ETA: 2s - loss: 0.2062 - accuracy: 0.95 - ETA: 2s - loss: 0.2099 - accuracy: 0.95 - ETA: 1s - loss: 0.2025 - accuracy: 0.95 - ETA: 1s - loss: 0.2031 - accuracy: 0.95 - ETA: 1s - loss: 0.2009 - accuracy: 0.95 - ETA: 1s - loss: 0.1939 - accuracy: 0.96 - ETA: 1s - loss: 0.1884 - accuracy: 0.96 - ETA: 1s - loss: 0.1936 - accuracy: 0.96 - ETA: 1s - loss: 0.1943 - accuracy: 0.96 - ETA: 1s - loss: 0.1935 - accuracy: 0.96 - ETA: 0s - loss: 0.2044 - accuracy

...

475/475 [==============================] - ETA: 3s - loss: 5.1951e-05 - accuracy: 1.00 - ETA: 3s - loss: 5.7407e-05 - accuracy: 1.00 - ETA: 3s - loss: 6.4085e-05 - accuracy: 1.00 - ETA: 3s - loss: 9.4195e-05 - accuracy: 1.00 - ETA: 3s - loss: 1.9698e-04 - accuracy: 1.00 - ETA: 3s - loss: 2.1110e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.8956e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.6807e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.5719e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.5540e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.4899e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.4049e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.3342e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.3033e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.2783e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.2527e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.2036e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.2001e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1718e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1348e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1343e-04 - accuracy

475/475 [==============================] - ETA: 3s - loss: 8.2009e-05 - accuracy: 1.00 - ETA: 3s - loss: 7.4502e-05 - accuracy: 1.00 - ETA: 3s - loss: 6.5353e-05 - accuracy: 1.00 - ETA: 3s - loss: 5.9939e-05 - accuracy: 1.00 - ETA: 3s - loss: 5.3004e-05 - accuracy: 1.00 - ETA: 3s - loss: 5.0874e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.8787e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.6562e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.7928e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.7693e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.0187e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.8154e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.0108e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.1463e-05 - accuracy: 1.00 - ETA: 1s - loss: 5.1607e-05 - accuracy: 1.00 - ETA: 1s - loss: 7.8514e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.8202e-05 - accuracy: 1.00 - ETA: 1s - loss: 1.7434e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.9542e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.8794e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.8304e-04 - accuracy

475/475 [==============================] - ETA: 3s - loss: 2.6262e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.5338e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.2180e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.2618e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.7781e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.8349e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.8259e-05 - accuracy: 1.00 - ETA: 2s - loss: 4.8607e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.2084e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.1219e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.0905e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.1908e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.4038e-05 - accuracy: 1.00 - ETA: 2s - loss: 5.4400e-05 - accuracy: 1.00 - ETA: 1s - loss: 5.2959e-05 - accuracy: 1.00 - ETA: 1s - loss: 5.5000e-05 - accuracy: 1.00 - ETA: 1s - loss: 6.8438e-05 - accuracy: 1.00 - ETA: 1s - loss: 8.5059e-05 - accuracy: 1.00 - ETA: 1s - loss: 8.2598e-05 - accuracy: 1.00 - ETA: 1s - loss: 8.2331e-05 - accuracy: 1.00 - ETA: 1s - loss: 8.1113e-05 - accuracy

475/475 [==============================] - ETA: 3s - loss: 5.8448e-05 - accuracy: 1.00 - ETA: 3s - loss: 3.8910e-05 - accuracy: 1.00 - ETA: 3s - loss: 4.6049e-05 - accuracy: 1.00 - ETA: 3s - loss: 6.7395e-05 - accuracy: 1.00 - ETA: 3s - loss: 6.5523e-05 - accuracy: 1.00 - ETA: 3s - loss: 7.4260e-05 - accuracy: 1.00 - ETA: 2s - loss: 8.0886e-05 - accuracy: 1.00 - ETA: 2s - loss: 1.0095e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0790e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0966e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0364e-04 - accuracy: 1.00 - ETA: 2s - loss: 9.8714e-05 - accuracy: 1.00 - ETA: 2s - loss: 9.4675e-05 - accuracy: 1.00 - ETA: 2s - loss: 9.4160e-05 - accuracy: 1.00 - ETA: 1s - loss: 1.0768e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.2175e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1741e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1384e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.1137e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.0942e-04 - accuracy: 1.00 - ETA: 1s - loss: 1.0757e-04 - accuracy

475/475 [==============================] - ETA: 3s - loss: 6.4673e-05 - accuracy: 1.00 - ETA: 3s - loss: 6.9888e-05 - accuracy: 1.00 - ETA: 3s - loss: 5.4569e-05 - accuracy: 1.00 - ETA: 3s - loss: 1.0847e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.4607e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.2772e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.1531e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.1519e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0540e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0965e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.0406e-04 - accuracy: 1.00 - ETA: 2s - loss: 9.7295e-05 - accuracy: 1.00 - ETA: 2s - loss: 9.8197e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.8755e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.5627e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.4339e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.4305e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.4394e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.4933e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.2717e-05 - accuracy: 1.00 - ETA: 1s - loss: 9.0821e-05 - accuracy